In [5]:
import sys
import os
import json
from pathlib import Path

import numpy as np

parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)
sys.path.append(parent_dir)

# Now you can import the module
import spade.utils.general_utils as gu
import spade.model.data_utils as du
from spade.utils.config_manager import ConfigManager
from spade.model.data_module import SpadeDataModule, SpadeData, get_data

/home/jovyan/ai-platform-volumes/zeroman0112/repos/spade_for_baemin_menu_kie


In [2]:
config_dir = f"{parent_dir}/configs"
data_dir = f"{parent_dir}/data"
# path_config_file = f"{config_dir}/cord.1.5layers.train.yaml"
path_config_file = f"{config_dir}/cord.1.5layers.train.yaml"

config_manager = ConfigManager(config_dir, path_config_file)
cfg = config_manager.cfg
cfg.path_data_folder = Path(data_dir)

In [3]:
fpath = cfg.path_data_folder / cfg.data_paths["train"]
raw_data = gu.load_jsonl(fpath, cfg["toy_data"], cfg["toy_size"])


In [4]:
# json_string = json.dumps(raw_data[0], indent=4)
# print(json_string)

data = raw_data[0]
print(data.keys())
text = data["text"]
fields = data["fields"]
field_rs = data["field_rs"]
vertical = data["vertical"]
label = np.array(data["label"])
coord = np.array(data["coord"])
image_size = data["img_sz"]

n_fields = len(fields)

dict_keys(['data_id', 'fields', 'field_rs', 'text', 'label', 'coord', 'vertical', 'img_sz', 'img_feature', 'img_url'])


In [5]:
print(f"n_fields: {len(fields)}")
print(f"n_texts: {len(text)}")

print(label.shape)
print(coord.shape)
print(len(vertical))
# print(coord)
print(image_size)

n_fields: 41
n_texts: 9
(2, 50, 9)
(9, 4, 2)
9
{'width': 1108, 'height': 1478}


In [6]:
print(coord)

[[[ 250  833]
  [ 325  834]
  [ 325  869]
  [ 250  868]]

 [[ 340  832]
  [ 434  833]
  [ 433  868]
  [ 339  867]]

 [[ 217  835]
  [ 227  834]
  [ 228  869]
  [ 217  869]]

 [[ 841  824]
  [ 954  824]
  [ 954  864]
  [ 841  864]]

 [[ 249  921]
  [ 342  920]
  [ 344  990]
  [ 249  992]]

 [[ 844  920]
  [ 959  920]
  [ 959  991]
  [ 844  991]]

 [[ 173 1045]
  [ 326 1044]
  [ 326 1111]
  [ 173 1114]]

 [[ 733 1041]
  [ 960 1041]
  [ 960 1117]
  [ 733 1117]]

 [[ 212  925]
  [ 230  926]
  [ 227  993]
  [ 213  994]]]


In [7]:
cfg.raw_data_input_type

'type1'

In [8]:
data_module = SpadeDataModule(cfg)

In [9]:
input_text = "안녕하세요. 바나나"
tokenizer = data_module.tokenizer
input_tokens = tokenizer.tokenize(input_text)

print(input_tokens)

['안', '##녕', '##하', '##세', '##요', '.', '바', '##나', '##나']


In [10]:
# pytorch Dataset object
train_data, dev_data = data_module._prepare_train_data()

# text token augmentation
input_tokens_aug = du.gen_augmented_text_tok1(
    train_data.token_pool, input_tokens, train_data.token_aug_param
)

print(input_tokens_aug)

# 이 정도 결과면 augmentation 안하는게 나을거 같은데..

type1
type1
['안', '##녕', '##하', '##세', '##요', '.', '##나', '##나', '##CT', '472']


In [11]:
_, coord = du.gen_augmented_coord(
    img=None, coord=coord, img_sz=image_size, coord_aug_params=train_data.coord_aug_params, clip_box_coord=True
)

In [12]:
print(np.array(coord).shape)

(9, 4, 2)


In [13]:
arr = np.array([1,2,3,0,1,2,3,0])
arr[arr != 0]

array([1, 2, 3, 1, 2, 3])

In [14]:
row_pointer = n_fields
rel_idx = 1 
label_sub = label[0]

print(label_sub[row_pointer])

np.where(label_sub[row_pointer] == rel_idx)

[0 1 0 0 0 0 0 0 0]


(array([1]),)

In [15]:
[] is not None

True

In [3]:
direc_vec = np.array([10, 10])
coords = [direc_vec] * 4


In [6]:
print(coords)
coords + direc_vec

[array([10, 10]), array([10, 10]), array([10, 10]), array([10, 10])]


array([[20, 20],
       [20, 20],
       [20, 20],
       [20, 20]])